In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import subprocess
from openai import OpenAI
import gdown
import re
import os
import openai
openai.api_key = ""

In [3]:
data = pd.read_csv("test.csv").drop(["Timestamp"], axis=1)

In [4]:
marks_dict = {
    'ID NUMBER' : data['ID NUMBER'],
    'MCQ1':0,
    'MCQ2':0,
    'MCQ3':0,
    'MCQ4':0,
    'MCQ5':0,
    'DQ1':0,
    'DQ2':0,
    'DQ3':0,
    'DQ4':0,
    'DQ5':0,
    'CQ':0,
    'Coding Reason': "",
    'Code Output': "",
    'Total':0,
}

marks = pd.DataFrame(marks_dict)

In [5]:
mcq_ans = ["","","","",""]
print("\nEnter the answers for the following question, copy paste the answer from the google form : \n")
for i in range(5):
    mcq_ans[i] = input(data.columns[i+1])

print("Thank you for answering")


Enter the answers for the following question, copy paste the answer from the google form : 

Thank you for answering


In [6]:
for question in range(1,6):
    for student in range(len(data)):
        if (data.iloc[student,question] == mcq_ans[question-1]):
            marks.iloc[student,question] +=1

In [7]:

def evaluvate_answer(question, answer):

  client = openai.OpenAI(api_key="")
  system_message = "You are Zoya, a question evaluvator for the subject of Object Oriented Programming with Java. You should be equipped with a good amount of knowledge on Java and OOP concepts. You will be given a question and a student submitted answer and your task is to assign the marks for the answer. Total marks for each question is 2 marks, and you can assign 0,0.5, 1, 1.5, 2 marks for the answer, you only have these 5 options to assign the marks. Your input will be a question, a answer and your output should be in the form marks which is float, like you should only give a float number which is the marks that you have assigned, and it must be one of 5 options i have given to you, only this should be your response, no other character should be there in your response.  "
  response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
      {"role": "system", "content": f"{system_message}"},
      {"role": "user", "content": f"Question: {question} Answer: {answer}\nDetermine the best way to fetch relevant information:"}
  ]
  )
  result = response.choices[0].message.content.strip()
  return result


In [8]:
for question in range(6,11):
    for student in range(len(data)):
        result = float(evaluvate_answer(data.columns[question],data.iloc[student,question]))
        marks.loc[student,marks.columns[question]] = result


C:\Users\nikhi\AppData\Local\Temp\ipykernel_18512\2922868336.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  marks.loc[student,marks.columns[question]] = result
C:\Users\nikhi\AppData\Local\Temp\ipykernel_18512\2922868336.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  marks.loc[student,marks.columns[question]] = result
C:\Users\nikhi\AppData\Local\Temp\ipykernel_18512\2922868336.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  marks.loc[student,marks.co

In [9]:

def get_java_file_name(java_code):
    # Find class name
    class_name_pattern = r"class\s+(\w+)\s*{"
    match = re.search(class_name_pattern, java_code)
    
    if match:
        class_name = match.group(1)
        
        # Check if class contains public static void main
        main_method_pattern = r"public\s+static\s+void\s+main\s*\("
        if re.search(main_method_pattern, java_code):
            # File name should be same as class name
            return class_name + ".java"
    
    return None


def save_java_file(url):
    file_id = url.split("=")[-1]
    download_url = f'https://drive.google.com/uc?export=download&id={file_id}'
    output = 'file.java'
    gdown.download(download_url, output, quiet=False)
    with open(output, 'r') as infile:
        java_code = infile.read()

    file_name = get_java_file_name(java_code)
    with open(file_name, 'w') as file:
        file.write(java_code)

    os.remove(output)

    return java_code, file_name

def run_java_file(url):

    java_code, file_name = save_java_file(url)
    output = "No Output"
    compile = subprocess.run(['javac', file_name], capture_output=True, text=True)

    if compile.returncode != 0:
        output = f"Compilation failed with the following message:\n{compile.stderr}"
    else:
        run = subprocess.run(['java', '-cp','.', file_name[:-5]], capture_output=True, text=True)

        if run.returncode == 0:
            output = run.stdout
        else: 
            output = f"Execution failed with the following message:\n{run.stderr}"
    return java_code, output
    


In [10]:

def evaluvate_code(question, java_code, output):

  client = openai.Client(api_key="")
  system_message = "You are Zoya, a question evaluvator for the subject of Object Oriented Programming with Java. You should be equipped with a good amount of knowledge on Java and OOP concepts. You will be given a coding question and a student submitted code and it's output, and your task is to assign the marks for the solution. You need to check wether the code is written correctly without any syntax errors or not  and the student code's output which will be given, you need to check if it's valid output for the question and code provided, you can assign upto 10 marks. If the output is an error you can decrease upto 3 marks based on the error. You need to also give the reason for why you decreased the marks, if they score 10 marks, reason should be no reason.Result is the marks obtained by the student after evaluvation. Your output should be in the form Result which is float after put a \n and then write the reason for why you decreased the marks, like you should only give a float number which is the final marks that you have assigned and then \n and then the reason. There must be only a single \n in the response and that one \n should be between the marks and the reason. No side headings needed in the response, no other character should be there in the reponse other than this. You strictly required to follow the format. Format: {result}\n{reason}. Examples output : 10 \n No reason"
  
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": f"{system_message}"},
        {"role": "user", "content": f"Question:{question}, Code:{java_code}, Student Output:{output}\n"}
    ]
  )
  result = response.choices[0].message.content.strip()
  return result


In [11]:
def process_code_result(str):
    s = str.split("\n")
    for i in range(2):
        s[i]=s[i].strip()
    return float(s[0]), s[1]

for i in range(len(data)):
    code, output = run_java_file(data.loc[i][-1])
    result  = evaluvate_code(data.columns[-1], code, output)
    mark, reason = process_code_result(result)
    marks.loc[i, 'CQ'] = mark
    marks.loc[i, 'Coding Reason'] = reason
    marks.loc[i, 'Code Output'] = output


C:\Users\nikhi\AppData\Local\Temp\ipykernel_18512\655947959.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  code, output = run_java_file(data.loc[i][-1])
Downloading...
From: https://drive.google.com/uc?export=download&id=11fhbneaCCEDTFV_iNzl7U2h5GZFbXjcr
To: c:\Users\nikhi\Desktop\Faculty Helper\file.java
100%|██████████| 832/832 [00:00<00:00, 833kB/s]
C:\Users\nikhi\AppData\Local\Temp\ipykernel_18512\655947959.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  code, output = run_java_file(data.loc[i][-1])
Downloading...
From: https://drive.google.com/uc?export=download&id=11gKniJMVAFw4JKwenbAIljFrYH8MPe3S
To: 

In [12]:
for student in range(len(data)):
    for question in range(1,12):
        marks.loc[student,"Total"]  += marks.loc[student, marks.columns[question]]

C:\Users\nikhi\AppData\Local\Temp\ipykernel_18512\1309175056.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  marks.loc[student,"Total"]  += marks.loc[student, marks.columns[question]]


In [35]:
marks = marks.sort_values("Total", ascending=False)

In [36]:
marks

,ID NUMBER,MCQ1,MCQ2,MCQ3,MCQ4,MCQ5,DQ1,DQ2,DQ3,DQ4,DQ5,CQ,Coding Reason,Code Output,Total
3,3,1,1,0,1,1,1.5,2.0,2,1.5,2.0,9.0,The student's code correctly demonstrates the ...,Area: 1.0\nArea: 12.0\nArea: 1.0\n,22.0
2,1,1,1,1,0,1,1.5,1.5,2,1.5,2.0,9.0,"The student's output is mostly correct, but th...",Area of r1: 1.0\nArea of r2: 20.0\nArea with w...,21.5
4,5,1,0,1,0,1,2.0,1.5,2,1.5,1.5,9.5,The student has demonstrated the usage of cons...,Area: 4.0\nArea: 21.0\nArea: 4.0\n,21.0
1,2,1,0,0,0,1,1.5,2.0,2,1.5,2.0,10.0,No reason,Area of r1: 1.0\nArea of r2: 20.0\nArea with w...,21.0
5,4,1,0,0,0,1,2.0,1.5,2,1.5,2.0,9.0,The student's code is mostly correct in terms ...,Area: 1.0\nArea: 12.0\nArea: 1.0\n,20.0
0,6,0,0,1,0,0,1.5,2.0,2,1.5,1.5,10.0,No reason,Area of r1: 1.0\nArea of r2: 20.0\nArea with w...,19.5


In [37]:
marks.to_csv("Result_Marks.csv")